In [1]:
"""
LWMの出力結果を用いて, 極値増大確率を算出する.

"""

import func
from csv import reader
import matplotlib.pyplot as plt
import math

def calc_p(past_rv, past_pro, f_rv, f_pro):
    """
    将来分布の中で, hsより大きい値を取る確率を合計する
    """
    P = 0
    for i in range(len(past_rv)):
        p = 0
        for j in range(len(f_rv)):
            if f_rv[j] > past_rv[i]:
                p += f_pro[j]
        P += past_pro[i] * p
    return P * 100

def cdf(rv, pro):
    Hs = []
    SUM = []
    sum = 1
    s = [(rv[i], pro[i]) for i in range(len(rv))]
    s.sort()
    for i in range(len(rv)):
        sum -= s[i][1]
        SUM.append(sum)
        Hs.append(s[i][0])
    return Hs, SUM

# 描画用の画像を用意
fig = plt.figure(figsize=(10, 14))

MODEL = ['KF', 'YS']
DATA_CNT = 100

for model_num in range(len(MODEL)):
    for num in range(2):
        model = MODEL[model_num]
        if num == 0:
            model = 'HPA_' + model
        else:
            model = 'HFA_' + model + '_c0'

        print(model)

        with open('../pot_csv(100)(thr=6)/' + model + '_POT_DATA.csv', 'r') as csv_file:
            csv_reader = reader(csv_file)
            POT = list(csv_reader)
        with open('../pot_csv(100)(thr=6)/' + model + '_POT_INDEX.csv', 'r') as csv_file:
            csv_reader = reader(csv_file)
            POT_INDEX = list(csv_reader)

        DATA = []  # [Hsのindex, Hs]の配列
        for i in range(len(POT)):
            for j in range(len(POT[i])):
                DATA.append([int(POT_INDEX[i][j]), float(POT[i][j])])
        
        # [Hsのindexがsortされる(小さい順)]
        DATA.sort()

        last_index = 0
        max_val = 0

        EVENT_POT = [[]]  # イベントごとのPOTを収納
        for data in DATA:
            if data[1] > 15.0:
                if data[0] > last_index + 24 * 7:
                    EVENT_POT.append([data[0], data[1]])
                    last_index = data[0]
                    max_val = data[1]
                else:
                    if data[1] > max_val:
                        EVENT_POT[-1] = [data[0], data[1]]
                        last_index = data[0]
                        max_val = data[1]
        
        EVENT_POT.pop(0)

        POT_DATA = []  # 空間最大波高
        for pot in EVENT_POT:
            POT_DATA.append(math.floor(pot[1] * 100) / 100)
        # 大きい順にsort
        POT_DATA.sort(reverse=True)
        
        if num == 0:
            past_rv, past_pro, x, s = func.lwm_gpd(data=POT_DATA, error=[0.005], thr=POT_DATA[-1], n=219143*79*79, n0=len(POT_DATA), con=1.0)
        else:
            f_rv, f_pro, x, s =  func.lwm_gpd(data=POT_DATA, error=[0.005], thr=POT_DATA[-1], n=219143*79*79, n0=len(POT_DATA), con=1.0)
        
    P = calc_p(past_rv, past_pro, f_rv, f_pro)
    print(P, '%の確率で極値増加')
    

HPA_KF
最尤推定 ξ: -0.384615384615385 σ: 8.242973735991995 RV: 34.770068637319284
HFA_KF_c0
最尤推定 ξ: -0.384615384615385 σ: 9.579701599703602 RV: 37.695729469743085
90.50025793755381 %の確率で極値増加
HPA_YS
最尤推定 ξ: -0.47267547267547305 σ: 7.311809435150502 RV: 29.703335299846856
HFA_YS_c0
最尤推定 ξ: -0.35871535871535914 σ: 6.991974642631229 RV: 32.0822668850619
89.50702541111518 %の確率で極値増加


<Figure size 720x1008 with 0 Axes>